# Imports:

In [1]:
# Import necessary libraries and the custom optimizer
import datetime
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import config
import dynamiqs as dq
import tensorflow as tf

import utilities as utl
import ET_utilities as etutl
from load_and_plot import plot_evolution
from optimizers import HardwareAwareOptimizer

%load_ext autoreload
%autoreload 2

config.update("jax_enable_x64", False)
tf.keras.backend.set_floatx('float32')
dq.set_progress_meter(False)

%matplotlib qt

# Set a global seed for reproducible results
seed_value = 44
tf.random.set_seed(seed_value)
np.random.seed(seed_value)


# Binomial X gate:

## Pre-training:

In [ ]:
# Load a pulse obtained from gradient descent
# It should be a numpy array of shape (4, n_pulse_points)
print("Loading a sample pulse for pre-training...")

directory = '/Users/saswataroy/OptimalControl/pulses/'
data = np.load(directory + 'bin_X_gate.npz')
amps = data["arr_0"]

ideal_pulse = amps

print("Sample pulse loaded.")

Loading a sample pulse for pre-training...


FileNotFoundError: [Errno 2] No such file or directory: 'C://Users//Fatem//SR_optctrl//Pulses//bin_X_gate.npz'

In [2]:
# DEFINE SIMULATION AND PHYSICS PARAMETERS
fourier_scale = 12
# Instantiate the main optimizer class
# Define the scaling factors for the output layer.
# These values multiply the standard Glorot-initialized weights.
# Format: (I_cavity, Q_cavity, I_qubit, Q_qubit)
init_scales = (8, 8, 20, 20)

# Simulation parameters
osc_drive_type = 'linear'
ncav = 15  # Hilbert space dimension for the cavity
ntr = 2    # Hilbert space dimension for the transmon

# Device parameters (in MHz, multiplied by 2*pi)
alpha = -165 * (2 * np.pi)
K = -0.0242 * (2 * np.pi)*0
chi = -3.649 * (2 * np.pi)
chi_prime = +0.039 * (2 * np.pi)

# Pulse parameters
T = 1  # Total pulse time in microseconds
ntsave = 1001
ntpulse = 1001
max_amp = 10 * jnp.pi * 2 # Max amplitude for the drives
max_freq = 50 # Max frequency component in MHz

# SETUP HAMILTONIAN, STATES, AND OPERATORS ---

# Operators
a, adag = dq.destroy(ncav), dq.create(ncav)
t, tdag = dq.destroy(ntr), dq.create(ntr)
idcav, idtr = dq.eye(ncav), dq.eye(ntr)

# Define Binomial codewords and errorwords
L0 = (dq.basis(ncav, 0) + dq.basis(ncav, 4)) / jnp.sqrt(2)
L1 = dq.basis(ncav, 2)

plus_X = dq.unit((L0 + L1) / jnp.sqrt(2)) 
minus_X = dq.unit((L0 - L1) / jnp.sqrt(2))
plus_Y = dq.unit((L0 + 1j*L1) / jnp.sqrt(2)) 
minus_Y = dq.unit((L0 - 1j*L1) / jnp.sqrt(2))

# Error words:
E0 = dq.basis(ncav, 3)
E1 = dq.basis(ncav, 1)
E_plus_X = dq.unit((E0 + E1) / jnp.sqrt(2))
E_minus_X = dq.unit((E0 - E1) / jnp.sqrt(2))
E_plus_Y = dq.unit((E0 + 1j*E1) / jnp.sqrt(2))
E_minus_Y = dq.unit((E0 - 1j*E1) / jnp.sqrt(2))

# Define initial and target states for the X-gate
psi0 = [dq.tensor(L0, dq.basis(ntr, 0)), 
        dq.tensor(L1, dq.basis(ntr, 0)),
        dq.tensor(plus_X, dq.basis(ntr, 0)),
        dq.tensor(minus_X, dq.basis(ntr, 0)),
        dq.tensor(plus_Y, dq.basis(ntr, 0)),
        dq.tensor(minus_Y, dq.basis(ntr, 0)),
        # dq.tensor(E0, dq.basis(ntr, 0)),
        # dq.tensor(E1, dq.basis(ntr, 0)),
        # dq.tensor(E_plus_X, dq.basis(ntr, 0)), 
        # dq.tensor(E_minus_X, dq.basis(ntr, 0)),
        # dq.tensor(E_plus_Y, dq.basis(ntr, 0)), 
        # dq.tensor(E_minus_Y, dq.basis(ntr, 0))
]

exp_ops = [dq.tensor(dq.proj(L1), dq.proj(dq.basis(ntr, 0))), 
           dq.tensor(dq.proj(L0), dq.proj(dq.basis(ntr, 0))),
           dq.tensor(dq.proj(plus_X), dq.proj(dq.basis(ntr, 0))), 
           dq.tensor(dq.proj(minus_X), dq.proj(dq.basis(ntr, 0))),
           dq.tensor(dq.proj(minus_Y), dq.proj(dq.basis(ntr, 0))), 
           dq.tensor(dq.proj(plus_Y), dq.proj(dq.basis(ntr, 0))),
        #    dq.tensor(dq.proj(E1), dq.proj(dq.basis(ntr, 0))), 
        #    dq.tensor(dq.proj(E0), dq.proj(dq.basis(ntr, 0))),
        #    dq.tensor(dq.proj(E_plus_X), dq.proj(dq.basis(ntr, 0))), 
        #    dq.tensor(dq.proj(E_minus_X), dq.proj(dq.basis(ntr, 0))),
        #    dq.tensor(dq.proj(E_minus_Y), dq.proj(dq.basis(ntr, 0))), 
        #    dq.tensor(dq.proj(E_plus_Y), dq.proj(dq.basis(ntr, 0)))
]

# Time arrays
tsave = jnp.linspace(0.0, T, ntsave, dtype = jnp.float32)
tpulse = jnp.linspace(0.0, T, ntpulse, dtype = jnp.float32)

# Static Hamiltonian
H0 = chi * dq.tensor(adag@a, tdag@t) + (K/2)*dq.tensor(adag@adag@a@a, idtr) + \
     (alpha/2)*dq.tensor(idcav, tdag@tdag@t@t) + (chi_prime/2)*dq.tensor(adag@adag@a@a, tdag@t)

# Create the master parameter dictionary
my_params = {'H0': H0, 
             'ncav': ncav, 
             'ntr': ntr, 
             'alpha': alpha, 
             'K': K, 
             'chi': chi,
             'chi_prime': chi_prime, 
             'T': T, 
             'ntsave': ntsave, 
             'ntpulse': ntpulse,
             'max_amp': max_amp, 
             'max_freq': max_freq, 
             'psi0': psi0, 
             'exp_ops': exp_ops, 
             'tsave': tsave, 
             'tpulse': tpulse, 
             'osc_drive': osc_drive_type}

In [3]:
optimizer = HardwareAwareOptimizer(my_params, 
                                   fourier_scale = fourier_scale,
                                   output_scales = init_scales)

Initializing Optimizer...
Neural network model created.


In [17]:
# Instantiate the main optimizer class
# optimizer = HardwareAwareOptimizer(my_params, fourier_scale=18)

# Pre-train the network on the GRADIENT descent generated pulse
optimizer.pre_train(
    ideal_pulse_arrays=ideal_pulse,
    time_array=my_params['tpulse'][:-1],
    epochs= 1000000,
    learning_rate=1e-3
)


--- Starting Pre-training ---


Pre-training:   0%|          | 2151/1000000 [01:19<10:15:40, 27.01it/s]


KeyboardInterrupt: 

In [14]:
print("Verifying the pre-trained pulse...")

# Prepare the time vector input for the model
time_array = my_params['tpulse'][:-1]
time_input = tf.constant(time_array, dtype=tf.float64)
time_input = tf.reshape(time_input, (1, len(time_array), 1))

# Generate a pulse from the trained network
# generated_pulse_tf = optimizer.model(time_input)
# generated_pulse_np = generated_pulse_tf.numpy()

generated_pulse_np = optimizer.generate_pulse()

# Reconstruct the ideal pulse into the same complex format for comparison
# I_c, Q_c, I_q, Q_q = [ideal_pulse[i] for i in range(4)]
# drive_c_ideal = I_c + 1j * Q_c"
# drive_q_ideal = I_q + 1j * Q_q
ideal_pulse_complex = amps

# Numerically compare by calculating the Mean Squared Error
mse = np.mean(np.abs(np.square(generated_pulse_np - ideal_pulse_complex)))
print(f"Mean Squared Error (NN vs Ideal): {mse:.6f}")

# Visually compare by plotting
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Cavity Drive Plot
axs[0].plot(time_array, np.real(ideal_pulse_complex[0, :]), 'b-', label='Ideal (Re)')
axs[0].plot(time_array, np.imag(ideal_pulse_complex[0, :]), 'r-', label='Ideal (Im)')
axs[0].plot(time_array, np.real(generated_pulse_np[0, :]), 'b--', label='NN (Re)')
axs[0].plot(time_array, np.imag(generated_pulse_np[0, :]), 'r--', label='NN (Im)')
axs[0].set_title('Cavity Drive Comparison')
axs[0].set_ylabel('Amplitude')
axs[0].legend()
axs[0].grid(True)

# Qubit Drive Plot
axs[1].plot(time_array, np.real(ideal_pulse_complex[1, :]), 'b-', label='Ideal (Re)')
axs[1].plot(time_array, np.imag(ideal_pulse_complex[1, :]), 'r-', label='Ideal (Im)')
axs[1].plot(time_array, np.real(generated_pulse_np[1, :]), 'b--', label='NN (Re)')
axs[1].plot(time_array, np.imag(generated_pulse_np[1, :]), 'r--', label='NN (Im)')
axs[1].set_title('Qubit Drive Comparison')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Amplitude')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

Verifying the pre-trained pulse...
Generating pulse from the trained model...
Mean Squared Error (NN vs Ideal): 14.315224


In [9]:
directory = 'C://Users//Fatem//SR_optctrl//Pulses//'
base_filename = 'pre_trained_bin_X_ET'

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

weights_filepath = directory + base_filename +'_'+ timestamp + '.weights.h5'
params_filepath = directory + base_filename + '_'+ timestamp + '.npz'

optimizer.save_model_weights(weights_filepath)
np.savez(params_filepath, **my_params)
print(weights_filepath)
print(params_filepath)

print("\nPre-trained model is saved and ready for fine-tuning.")

Model weights saved successfully to C://Users//Fatem//SR_optctrl//Pulses//pre_trained_bin_X_ET_2025-09-11_08-45-09.weights.h5
C://Users//Fatem//SR_optctrl//Pulses//pre_trained_bin_X_ET_2025-09-11_08-45-09.weights.h5
C://Users//Fatem//SR_optctrl//Pulses//pre_trained_bin_X_ET_2025-09-11_08-45-09.npz

Pre-trained model is saved and ready for fine-tuning.


In [ ]:
print(str(weights_filepath))

## Offline Training:

In [ ]:
# Load the optimizer
directory = 'C://Users//Fatem//SR_optctrl//Pulses//'

optimizer = HardwareAwareOptimizer(my_params, 
                                   fourier_scale = fourier_scale,
                                   output_scales = init_scales)
# optimizer.load_model_weights(directory + 'pre_trained_bin_X_2025-09-02_10-07-09.weights.h5')
# optimizer.load_model_weights(directory + 'pre_trained_bin_X_ET_2025-09-11_08-45-09.weights.h5')
print('\n Model loaded, pre-training skipped')

Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//pre_trained_bin_X_ET_2025-09-11_08-45-09.weights.h5

 Model loaded, pre-training skipped


In [4]:
def loss_calculator_func(amps, params):

    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[0,:])), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[0,:])), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[1,:])), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[1,:])), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.sesolve(H, psi0, tsave, exp_ops = exp_ops, options = options, method = solver)
    
    avg_gate_fidelity_loss = utl.compute_fidelity_loss(evo_result)

    loss = 0
    cnt=0

    loss += 1*avg_gate_fidelity_loss
    cnt+=1

    return loss

In [19]:
loss_calculator_func(optimizer.generate_pulse(), my_params)

Generating pulse from the trained model...


Array(0.95285344, dtype=float32)

In [20]:
optimizer.fine_tune_simulation(
    loss_calculator_func=loss_calculator_func,
    epochs=50000,
    learning_rate=1e-3,
    update_plot_every = 10
)


--- Starting Simulation Fine-tuning ---


Sim-tuning:   0%|          | 248/50000 [13:14<44:15:43,  3.20s/it]


Target fidelity loss of 0.001 reached at epoch 248. Stopping fine-tuning.
Simulation fine-tuning finished. Final Fidelity Loss: 0.000997


In [22]:
# Save trained model weights

directory = '/Users/saswataroy/OptimalControl/pulses/'
base_filename = 'offline_trained_bin_X'

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

weights_filepath = directory + base_filename +'_'+ timestamp + '.weights.h5'
params_filepath = directory + base_filename + '_'+ timestamp + '.npz'

optimizer.save_model_weights(weights_filepath)
np.savez(params_filepath, **my_params)
print(repr(weights_filepath))
print(repr(params_filepath))

print("\nTrained model is saved and ready for hardware SPSA tuning")

Model weights saved successfully to /Users/saswataroy/OptimalControl/pulses/offline_trained_bin_X_2025-10-14_13-51-54.weights.h5
'/Users/saswataroy/OptimalControl/pulses/offline_trained_bin_X_2025-10-14_13-51-54.weights.h5'
'/Users/saswataroy/OptimalControl/pulses/offline_trained_bin_X_2025-10-14_13-51-54.npz'

Trained model is saved and ready for hardware SPSA tuning


### Robustness training:

In [ ]:
# THiS Still has a wrong calculation of fidelity (not taking the diagonal elements)
def robust_loss_calculator(amps, params):
    """
    Calculates a weighted average of a central fidelity and a robust fidelity
    over a distribution of Hamiltonian parameters in a single, efficient simulation.
    """
    # Setup Operators and Parameter Variations ---
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav, idtr = dq.eye(params['ncav']), dq.eye(params['ntr'])

    # Assumes the first entry in each list is the nominal (central) value - MHz without 2pi
    chi_variations = jnp.array([-3.649, -3.649 - 0.04, -3.649 + 0.04], dtype = jnp.float32)*jnp.pi*2
    # kerr_variations = jnp.array([-0.0242, -0.0242 - 0.005, -0.0242 + 0.005], dtype = jnp.float32)*jnp.pi*2
    qb_dets_variations = jnp.array([0, -0.04, +0.04] , dtype = jnp.float32)*jnp.pi*2
    
    # Get other static parameters
    # pi_32 = jnp.float32(jnp.pi)
    alpha = jnp.float32(params['alpha'])
    chi_prime = jnp.float32(params['chi_prime'])
    K = jnp.float32(params['K'])
    psi0 = params['psi0']
    tsave = params['tsave']
    exp_ops = params['exp_ops']

    # Build the Time-Dependent Drive Hamiltonian (once) ---
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive'] == 'squeeze':
        osc_pow = 2

    # Build the Batch of Hamiltonians ---
    H0_batch_list = []
    qb_drive_amps_list = []
    for chi_mhz in chi_variations:
        # for kerr_mhz in kerr_variations:
        for qb_det in qb_dets_variations:

            chi = chi_mhz 
            # K = kerr_mhz * (2 * jnp.pi)
            H0_sample = (chi * dq.tensor(adag @ a, tdag @ t) +
                        (K / 2) * dq.tensor(adag @ adag @ a @ a, idtr) +
                        (alpha / 2) * dq.tensor(idcav, tdag @ tdag @ t @ t) +
                        (chi_prime / 2) * dq.tensor(adag @ adag @ a @ a, tdag @ t))
            
            H0_batch_list.append(H0_sample)

            qb_det_phi = qb_det * params['tpulse'][:-1]
            det_qb_drive = amps[1,:] * jnp.exp(-1j*qb_det_phi)
            qb_drive_amps_list.append(det_qb_drive)
                

    # Perform a batch simulation ---
    H0_batch = dq.stack(H0_batch_list)
    qb_drive_amps_batch = jnp.stack(qb_drive_amps_list)

    osc_amps = amps[0,:]
    osc_amps_batched = jnp.broadcast_to(osc_amps, qb_drive_amps_batch.shape)

    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(osc_amps_batched)), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(osc_amps_batched)), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(qb_drive_amps_batch)), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(qb_drive_amps_batch)), -1j * dq.tensor(idcav, (t - tdag)))
    
    H_total_batch = H0_batch + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter=None)
    solver = dq.method.Tsit5(max_steps=int(1e9))
    batch_evo_result = dq.sesolve(
        H_total_batch, psi0, tsave, exp_ops=exp_ops, options=options, method=solver
    )

    # The final expectation values for all simulations in the batch
    all_expects_final = batch_evo_result.expects[..., -1]
    
    # The central fidelity is from the first simulation in the batch (index 0)
    central_fidelity = jnp.mean(all_expects_final[0, :, :])
    
    # The robust fidelity is the average over all simulations in the batch
    robust_fidelity = jnp.mean(all_expects_final)

    # Combine and Return the Final Loss ---
    robustness_weight = 0.0 
    weighted_fidelity = (central_fidelity + robustness_weight * robust_fidelity) / (1 + robustness_weight)
     
    if central_fidelity > 0.999:
        total_infidelity = 1.0 - central_fidelity
    else:
        total_infidelity = 1.0 - weighted_fidelity

    return jnp.real(total_infidelity)

In [27]:
def robust_loss_calculator_static(amps, params):
    """
    Calculates robust loss by batching over only static Hamiltonian
    parameters (chi and self-Kerr). The drive Hamiltonian is common to all simulations.
    """
    # Operators and Parameter Variations
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav, idtr = dq.eye(params['ncav']), dq.eye(params['ntr'])

    # Define variations for static parameters in MHz
    chi_variations_mhz = jnp.array([-3.649, -3.649 - 0.05, -3.649 + 0.05], dtype=jnp.float32)*2*jnp.pi
    kerr_variations_mhz = jnp.array([-0.0242, -0.0242 - 0.005, -0.0242 + 0.005], dtype=jnp.float32)*2*jnp.pi
    
    # Get constant parameters from the params dictionary
    alpha = jnp.float32(params['alpha'])
    chi_prime = jnp.float32(params['chi_prime'])
    # psi0, tsave, exp_ops = params['psi0'], params['tsave'], params['exp_ops']

    # Build the common Time-Dependent Drive Hamiltonian (once)
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    else:
        osc_pow = 2
    
    Hcr = dq.pwc(params['tpulse'], jnp.real(amps[0,:]), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.imag(amps[0,:]), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(amps[1,:]), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.imag(amps[1,:]), -1j * dq.tensor(idcav, (t - tdag)))
    H_drive = Hcr + Hci + Htr + Hti

    #  Build a Batch of the static Hamiltonians 
    H0_batch_list = []
    for chi_mhz in chi_variations_mhz:
        for kerr_mhz in kerr_variations_mhz:
            chi = chi_mhz 
            K = kerr_mhz 
            H0_sample = (chi * dq.tensor(adag @ a, tdag @ t) +
                         (K / 2) * dq.tensor(adag @ adag @ a @ a, idtr) +
                         (alpha / 2) * dq.tensor(idcav, tdag @ tdag @ t @ t) +
                         (chi_prime / 2) * dq.tensor(adag @ adag @ a @ a, tdag @ t))
            H0_batch_list.append(H0_sample)

    # Stack the static Hamiltonians and add the common drive 
    H0_batch = dq.stack(H0_batch_list)
    H_total_batch = H0_batch + H_drive

    # Perform Simulation and Calculate Loss 
    options = dq.Options(progress_meter=None)
    solver = dq.method.Tsit5(max_steps=int(1e9))
    batch_evo_result = dq.sesolve(
        H_total_batch, psi0, tsave, exp_ops=exp_ops, options=options, method=solver
    )

    all_expects_final = batch_evo_result.expects[:,:,:,-1].real
    num_h, num_psi = (all_expects_final.shape)[0:2]
    central_fidelity = sum(all_expects_final[0, i, i] for i in range(num_psi)) / num_psi
    robust_fidelity = sum(sum(all_expects_final[:, i, i] for i in range(num_psi))) / (num_psi * num_h)
    
    robustness_weight=0.5
    weighted_fidelity = (central_fidelity + robustness_weight * robust_fidelity) / (1 + robustness_weight)
    
    if central_fidelity>0.999:
        total_infidelity = 1 - central_fidelity
    else:
        total_infidelity = 1.0 - weighted_fidelity
    
    return jnp.real(total_infidelity)

#### Time-dependent batched variations:

In [10]:
def build_robustness_tensors(params):
    # Operators and Parameter Variations
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav, idtr = dq.eye(params['ncav']), dq.eye(params['ntr'])

    # Define variations in MHz
    chi_variations_mhz = jnp.array([-3.649, -3.649 - 0.005, -3.649 + 0.005], dtype=jnp.float32)
    qb_dets_variations_mhz = jnp.array([0, -0.04, +0.04], dtype=jnp.float32)

    # Get constant parameters
    alpha = jnp.float32(params['alpha'])
    chi_prime = jnp.float32(params['chi_prime'])
    K = jnp.float32(params['K'])
    pi_32 = jnp.float32(jnp.pi)

    # Build a Batch of the static Hamiltonians and detuning phasors
    H0_batch_list = []
    phasor_batch_list = []
    
    for chi_mhz in chi_variations_mhz:
        for qb_det_mhz in qb_dets_variations_mhz:
            chi = chi_mhz * (2 * pi_32)
            H0_sample = (chi * dq.tensor(adag @ a, tdag @ t) +
                         (K / 2) * dq.tensor(adag @ adag @ a @ a, idtr) +
                         (alpha / 2) * dq.tensor(idcav, tdag @ tdag @ t @ t) +
                         (chi_prime / 2) * dq.tensor(adag @ adag @ a @ a, tdag @ t))
            H0_batch_list.append(H0_sample)

            qb_det = qb_det_mhz * (2 * pi_32)
            qb_det_phase = qb_det * params['tpulse'][:-1]
            phasor = jnp.exp(-1j * qb_det_phase)
            phasor_batch_list.append(phasor)

    # Return a dictionary of the pre-calculated, stacked tensors
    return {
        'H0_batch': dq.stack(H0_batch_list),
        'phasor_batch': jnp.stack(phasor_batch_list)
    }

robustness_tensors = build_robustness_tensors(my_params)
my_params.update(robustness_tensors)

In [11]:
optimizer = HardwareAwareOptimizer(my_params)

# Load the optimizer
directory = 'C://Users//Fatem//SR_optctrl//Pulses//'

optimizer = HardwareAwareOptimizer(my_params)
optimizer.load_model_weights(directory + 'pre_trained_bin_X_2025-09-02_10-07-09.weights.h5')
print('\n Model loaded, pre-training skipped')

Initializing Optimizer...
Neural network model created.
Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//pre_trained_bin_X_2025-09-02_10-07-09.weights.h5

 Model loaded, pre-training skipped


In [12]:
def robust_loss_calculator(amps, params, robustness_weight=0.25):
    # Operators and constant parameters
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav, idtr = dq.eye(params['ncav']), dq.eye(params['ntr'])
    psi0, tsave, exp_ops = params['psi0'], params['tsave'], params['exp_ops']
    
    # Fetch pre-calculated tensors from the params dictionary
    H0_batch = params['H0_batch']
    phasor_batch = params['phasor_batch']
    
    # Build the time-dependent part of the Hamiltonian
    if params['osc_drive'] == 'linear': osc_pow = 1
    else: osc_pow = 2
    
    # Apply the pre-calculated phase to the qubit drive
    det_qb_drive_batch = amps[1,:] * phasor_batch  # Broadcasting (B, N) = (N,) * (B, N)

    # Broadcast the common cavity drive to match the batch shape
    cavity_drive_amps = amps[0,:]
    cavity_drive_amps_batch = jnp.broadcast_to(cavity_drive_amps, det_qb_drive_batch.shape)

    # Construct the full batched Hamiltonian
    Hcr = dq.pwc(params['tpulse'], jnp.real(cavity_drive_amps_batch), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.imag(cavity_drive_amps_batch), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(det_qb_drive_batch), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.imag(det_qb_drive_batch), -1j * dq.tensor(idcav, (t - tdag)))

    H_total_batch = H0_batch + Hcr + Hci + Htr + Hti

    # Perform Simulation and Calculate Loss
    options = dq.Options(progress_meter=None)
    solver = dq.method.Tsit5(max_steps=int(1e9))
    batch_evo_result = dq.sesolve(
        H_total_batch, psi0, tsave, exp_ops=exp_ops, options=options, method=solver
    )

    all_expects_final = batch_evo_result.expects[...,-1].real
    num_psi = all_expects_final.shape[1]

    # Vectorized fidelity calculation
    traces = jnp.trace(all_expects_final, axis1=1, axis2=2)
    central_fidelity = traces[0] / num_psi
    robust_fidelity = jnp.mean(traces) / num_psi
    
    weighted_fidelity = (central_fidelity + robustness_weight * robust_fidelity) / (1 + robustness_weight)
    
    if central_fidelity > 0.999:
        total_infidelity = 1 - central_fidelity
    else:
        total_infidelity = 1.0 - weighted_fidelity
    
    return jnp.real(total_infidelity)

#### static batched hamiltonian:

In [53]:
def build_static_hamiltonian_batch(params):
    # Operators and Parameter Variations
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav, idtr = dq.eye(params['ncav']), dq.eye(params['ntr'])

    # Define variations for static parameters in MHz
    chi_variations_mhz = jnp.array([-3.649, -3.649 - 0.03, -3.649 + 0.03], dtype=jnp.float32)
    kerr_variations_mhz = jnp.array([-0.0242, -0.0242 - 0.005, -0.0242 + 0.005], dtype=jnp.float32)
    
    # Get constant parameters
    alpha = jnp.float32(params['alpha'])
    chi_prime = jnp.float32(params['chi_prime'])
    pi_32 = jnp.float32(jnp.pi)
    
    # Build a Batch of the static Hamiltonians
    H0_batch_list = []
    for chi_mhz in chi_variations_mhz:
        for kerr_mhz in kerr_variations_mhz:
            chi = chi_mhz * (2 * pi_32)
            K = kerr_mhz * (2 * pi_32)
            H0_sample = (chi * dq.tensor(adag @ a, tdag @ t) +
                         (K / 2) * dq.tensor(adag @ adag @ a @ a, idtr) +
                         (alpha / 2) * dq.tensor(idcav, tdag @ tdag @ t @ t) +
                         (chi_prime / 2) * dq.tensor(adag @ adag @ a @ a, tdag @ t))
            H0_batch_list.append(H0_sample)
            
    return dq.stack(H0_batch_list)

my_params['H0_batch'] = build_static_hamiltonian_batch(my_params)
optimizer = HardwareAwareOptimizer(my_params)

Initializing Optimizer...
Neural network model created.


In [54]:
# Load the optimizer
directory = 'C://Users//Fatem//SR_optctrl//Pulses//'

optimizer = HardwareAwareOptimizer(my_params)
optimizer.load_model_weights(directory + 'pre_trained_bin_X_2025-09-02_10-07-09.weights.h5')
print('\n Model loaded, pre-training skipped')

Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//pre_trained_bin_X_2025-09-02_10-07-09.weights.h5

 Model loaded, pre-training skipped


In [32]:
# Load the optimizer
directory = 'C://Users//Fatem//SR_optctrl//Pulses//'

optimizer = HardwareAwareOptimizer(my_params)
optimizer.load_model_weights(directory + 'simulation_trained_bin_X_robust_2025-09-04_18-22-06.weights.h5')
print('\n Model loaded, pre-training skipped')

Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_robust_2025-09-04_18-22-06.weights.h5

 Model loaded, pre-training skipped


In [55]:

def robust_loss_calculator_static(amps, params):
    # Operators and constant parameters
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav, idtr = dq.eye(params['ncav']), dq.eye(params['ntr'])
    psi0, tsave, exp_ops = params['psi0'], params['tsave'], params['exp_ops']
    
    # Build the common Time-Dependent Drive Hamiltonian
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    else:
        osc_pow = 2
    
    Hcr = dq.pwc(params['tpulse'], jnp.real(amps[0,:]), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.imag(amps[0,:]), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(amps[1,:]), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.imag(amps[1,:]), -1j * dq.tensor(idcav, (t - tdag)))
    H_drive = Hcr + Hci + Htr + Hti

    # Fetch the pre-calculated static Hamiltonian batch and add the common drive
    H0_batch = params['H0_batch']
    H_total_batch = H0_batch + H_drive

    # Perform Simulation and Calculate Loss
    options = dq.Options(progress_meter=None)
    solver = dq.method.Tsit5(max_steps=int(1e9))
    batch_evo_result = dq.sesolve(
        H_total_batch, psi0, tsave, exp_ops=exp_ops, options=options, method=solver
    )

    all_expects_final = batch_evo_result.expects[...,-1].real
    num_psi = all_expects_final.shape[1]

    # Vectorized fidelity calculation
    # Takes the trace (sum of diagonals) for each matrix in the batch
    traces = jnp.trace(all_expects_final, axis1=1, axis2=2)
    
    central_fidelity = traces[0] / num_psi
    robust_fidelity = jnp.mean(traces) / num_psi
    
    robustness_weight=0.05
    weighted_fidelity = (central_fidelity + robustness_weight * robust_fidelity) / (1 + robustness_weight)
    
    if central_fidelity > 0.999:
        total_infidelity = 1 - central_fidelity
    else:
        total_infidelity = 1.0 - weighted_fidelity
    
    return jnp.real(total_infidelity)

In [13]:
optimizer.fine_tune_simulation(
    loss_calculator_func= robust_loss_calculator,
    epochs=60000,
    learning_rate=2.5e-4,
    update_plot_every = 25
)


--- Starting Simulation Fine-tuning ---


Sim-tuning:   0%|          | 28/60000 [09:09<198:37:20, 11.92s/it] 

: 

In [58]:
# Save trained model weights

directory = 'C://Users//Fatem//SR_optctrl//Pulses//'
base_filename = 'simulation_trained_bin_X_robust_ordinary'

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

weights_filepath = directory + base_filename +'_'+ timestamp + '.weights.h5'
params_filepath = directory + base_filename + '_'+ timestamp + '.npz'

optimizer.save_model_weights(weights_filepath)
np.savez(params_filepath, **my_params)
print(repr(weights_filepath))
print(repr(params_filepath))

print("\nTrained model is saved and ready for hardware SPSA tuning")

Model weights saved successfully to C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_robust_ordinary_2025-09-06_22-45-17.weights.h5
'C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_robust_ordinary_2025-09-06_22-45-17.weights.h5'
'C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_robust_ordinary_2025-09-06_22-45-17.npz'

Trained model is saved and ready for hardware SPSA tuning


### ET training: 

In [17]:
def et_loss_calculator_func(amps, params):

    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[0,:])), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[0,:])), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[1,:])), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[1,:])), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.sesolve(H, psi0, tsave, exp_ops = exp_ops, options = options, method = solver)
    
    avg_gate_fidelity_loss = utl.compute_fidelity_loss(evo_result)

    ET_fidelity_loss = etutl.compute_ET_fidelity_loss(evo_result, a, idtr)
    
    vel_var_loss = etutl.velocity_variance_loss(evo_result)

    loss = avg_gate_fidelity_loss + 0.1*ET_fidelity_loss + 0.0*vel_var_loss

    # print(avg_gate_fidelity_loss.value)

    return loss

In [18]:
optimizer.fine_tune_simulation(
    loss_calculator_func=et_loss_calculator_func,
    epochs=30000,
    learning_rate=5e-4,
    update_plot_every = 50
)


--- Starting Simulation Fine-tuning ---


Sim-tuning:   0%|          | 2/30000 [01:28<305:01:27, 36.61s/it]

: 

## SPSA testing in simulation:

In [39]:
# Load model weights

directory = '/Users/saswataroy/OptimalControl/pulses/'
optimizer = HardwareAwareOptimizer(my_params, 
                                   fourier_scale = fourier_scale,
                                   output_scales = init_scales)
optimizer.load_model_weights(directory + 'offline_trained_bin_X_2025-10-14_13-51-54.weights.h5')
print('\nModel loaded, proceed to SPSA hardware tuning')

Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from /Users/saswataroy/OptimalControl/pulses/offline_trained_bin_X_2025-10-14_13-51-54.weights.h5

Model loaded, proceed to SPSA hardware tuning


In [40]:
loss_calculator_func(optimizer.generate_pulse(), my_params)

Generating pulse from the trained model...


Array(0.00103097, dtype=float32)

In [44]:
my_test_params = my_params.copy()

# Device parameters (in MHz, multiplied by 2*pi)
alpha = -165 * (2 * np.pi)
K = (-0.0242  + 0.005) * (2 * np.pi) * 0
chi = (-3.649 + 0.01) * (2 * np.pi)
chi_prime = (+0.039 + 0.0) * (2 * np.pi)

# Detuning on qubit and cavity drives
det_osc = 0.02*(2*np.pi)*1
det_qb = -0.02*(2*np.pi)*1

my_test_params['H0'] = chi * dq.tensor(adag@a, tdag@t) + (K/2)*dq.tensor(adag@adag@a@a, idtr) + \
        (alpha/2)*dq.tensor(idcav, tdag@tdag@t@t) + (chi_prime/2)*dq.tensor(adag@adag@a@a, tdag@t)

def spsa_test_loss_calculator(amps, params):
    """
    Test loss calculator for SPSA testing
    """
    
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # Add noise to the drive
    iq_gain_errors = 1.0 + np.random.uniform(-0.001, 0.001, size=(4,))
    amps_real_noisy = np.real(amps) * np.array([[iq_gain_errors[0]], [iq_gain_errors[2]]])
    amps_imag_noisy = np.imag(amps) * np.array([[iq_gain_errors[1]], [iq_gain_errors[3]]])

    # noisy_amps = amps_real_noisy + 1j * amps_imag_noisy
    noisy_amps = amps

    # Add detuning to the drives
    qb_det_phi = det_qb * params['tpulse'][:-1]
    osc_det_phi = det_osc * params['tpulse'][:-1]
    det_osc_drive = noisy_amps[0,:] * jnp.exp(-1j*osc_det_phi)
    det_qb_drive = noisy_amps[1,:] * jnp.exp(-1j*qb_det_phi)

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_osc_drive)), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_osc_drive)), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_qb_drive)), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_qb_drive)), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.sesolve(H, psi0, tsave, exp_ops = exp_ops, options = options, method = solver)
    
    # add measurement noise to fidelity calculation
    measurement_noise = 0.5*np.random.uniform(-0.01, 0.01)
    avg_gate_fidelity_loss = np.clip(utl.compute_fidelity_loss(evo_result) + measurement_noise, 0, 1)

    loss = 0
    cnt=0

    loss += 1*avg_gate_fidelity_loss
    cnt+=1

    return loss

T1q = 70
T2q = 30
Tphiq = (1/T2q - 1/(2*T1q))**(-1)

T1c = 200
T2c = 280
Tphic = (1/T2c - 1/(2*T1c))**(-1)

c_ops = [
        jnp.sqrt(1/T1q)*dq.tensor(idcav, t),
        jnp.sqrt(1/Tphiq)*dq.tensor(idcav, tdag @ t),
        jnp.sqrt(1/T1c)*dq.tensor(a, idtr), 
        jnp.sqrt(1/Tphic)*dq.tensor(adag @ a, idtr),
        ]

exp_state = [dq.tensor(dq.basis(ncav,2), dq.basis(ntr,0))]
exp_code_dm_list = [psi @ psi.dag() for psi in exp_state]
exp_code_dm = dq.stack(exp_code_dm_list)

my_params['T1q'] = T1q
my_params['T2q'] = T2q
my_params['T1c'] = T1c
my_params['T2c'] = T2c
my_params['c_ops'] = c_ops
my_params['exp_state'] = exp_state
my_params['exp_code_dm'] = exp_code_dm

def spsa_test_loss_calculator_mesolve(amps, params):
    """
    Test loss calculator for SPSA testing
    """
    
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # Add noise to the drive
    # iq_gain_errors = 1.0 + np.random.uniform(-0.001, 0.001, size=(4,))
    # amps_real_noisy = np.real(amps) * np.array([[iq_gain_errors[0]], [iq_gain_errors[2]]])
    # amps_imag_noisy = np.imag(amps) * np.array([[iq_gain_errors[1]], [iq_gain_errors[3]]])

    # noisy_amps = amps_real_noisy + 1j * amps_imag_noisy
    noisy_amps = amps

    # Add detuning to the drives
    qb_det_phi = det_qb * params['tpulse'][:-1]
    osc_det_phi = det_osc * params['tpulse'][:-1]
    det_osc_drive = noisy_amps[0,:] * jnp.exp(-1j*osc_det_phi)
    det_qb_drive = noisy_amps[1,:] * jnp.exp(-1j*qb_det_phi)

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_osc_drive)), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_osc_drive)), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_qb_drive)), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_qb_drive)), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.mesolve(H, rho0=psi0, tsave=tsave, jump_ops = c_ops, exp_ops = exp_ops, options = options, method = solver)
    
    # finals = evo_result.states[:,-1]
    # fid = jnp.mean(jax.vmap(lambda r, t: jnp.abs(dq.trace(r@t)), in_axes=(0,0))(finals, exp_code_dm))

   # add measurement noise to fidelity calculation
    measurement_noise = 0*np.random.uniform(-0.01, 0.01)
    avg_gate_fidelity_loss = np.clip(utl.compute_fidelity_loss(evo_result) + measurement_noise, 0, 1)

    loss = 0
    cnt=0

    loss += 1*avg_gate_fidelity_loss
    cnt+=1

    return loss

def loss_calculator_func(amps, params):

    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # time-dependent Hamiltonian
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[0,:])), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[0,:])), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[1,:])), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[1,:])), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.sesolve(H, psi0, tsave, exp_ops = exp_ops, options = options, method = solver)
    
    avg_gate_fidelity_loss = utl.compute_fidelity_loss(evo_result)

    loss = 0
    cnt=0

    loss += 1*avg_gate_fidelity_loss
    cnt+=1

    return loss

def loss_calculator_func_mesolve(amps, params):

    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # time-dependent Hamiltonian
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[0,:])), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[0,:])), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[1,:])), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[1,:])), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.mesolve(H, rho0=psi0, tsave=tsave, jump_ops = c_ops, exp_ops = exp_ops, options = options, method = solver)
    
    avg_gate_fidelity_loss = utl.compute_fidelity_loss(evo_result)

    loss = 0
    cnt=0

    loss += 1*avg_gate_fidelity_loss
    cnt+=1

    return loss

def fourier_transform_rows(amps_time: np.ndarray, dt: float = 1e-9):
    """
    Compute the FFT of each row of a 2×N time-domain array.
    
    Parameters
    ----------
    amps_time : np.ndarray, shape (2, N)
        Time-domain complex amplitudes (row 0 = oscillator, row 1 = qubit).
    dt : float
        Time step in seconds between samples (default 1e-9 s).
    
    Returns
    -------
    freqs : np.ndarray, shape (N,)
        Frequency bins in Hz, as returned by np.fft.fftfreq.
    eps_freq : np.ndarray, shape (2, N)
        Complex amplitudes in the frequency domain.
    """
    N = amps_time.shape[1]
    freqs = np.fft.fftfreq(N, d=dt)
    eps_freq = np.fft.fft(amps_time, axis=1)
    return freqs, eps_freq

def inverse_fourier_transform_rows(eps_mod: np.ndarray):
    """
    Inverse FFT to go back to a 2×N time-domain waveform.
    
    Parameters
    ----------
    eps_mod : np.ndarray, shape (2, N)
        Frequency-domain amplitudes (modified).
    
    Returns
    -------
    amps_time_mod : np.ndarray, shape (2, N)
        Complex time-domain amplitudes, at the original 1 ns sampling.
    """
    amps_time_mod = np.fft.ifft(eps_mod, axis=1)
    return amps_time_mod

def dispersion_tuning(amps: np.ndarray,
                        b_osc: float, b_qb: float,
                        tau_osc: float, tau_qb: float,
                        dt: float = 1e-9) -> np.ndarray:
    freqs, eps_freq = fourier_transform_rows(amps, dt=dt)

    ω = 2 * np.pi * freqs    
    f_osc = 1 + b_osc * ω * np.exp(1j * ω * tau_osc)
    f_qb  = 1 + b_qb  * ω * np.exp(1j * ω * tau_qb)
    
    factors = np.vstack([f_osc, f_qb])  

    eps_mod = eps_freq * factors       

    amps_mod = inverse_fourier_transform_rows(eps_mod)
    return amps_mod  


def spsa_test_loss_calculator_dispersion_mesolve(amps, params):
    """
    Test loss calculator for SPSA testing
    """
    
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # Add noise to the drive
    # iq_gain_errors = 1.0 + np.random.uniform(-0.001, 0.001, size=(4,))
    # amps_real_noisy = np.real(amps) * np.array([[iq_gain_errors[0]], [iq_gain_errors[2]]])
    # amps_imag_noisy = np.imag(amps) * np.array([[iq_gain_errors[1]], [iq_gain_errors[3]]])

    # noisy_amps = amps_real_noisy + 1j * amps_imag_noisy

    amps = dispersion_tuning(amps, 0, 0.001*1e-6, 0, 0)
    noisy_amps = amps

    # Add detuning to the drives
    qb_det_phi = det_qb * params['tpulse'][:-1]
    osc_det_phi = det_osc * params['tpulse'][:-1]
    det_osc_drive = noisy_amps[0,:] * jnp.exp(-1j*osc_det_phi)
    det_qb_drive = noisy_amps[1,:] * jnp.exp(-1j*qb_det_phi)

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_osc_drive)), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_osc_drive)), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_qb_drive)), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_qb_drive)), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.mesolve(H, rho0=psi0, tsave=tsave, jump_ops = c_ops, exp_ops = exp_ops, options = options, method = solver)
    
    # finals = evo_result.states[:,-1]
    # fid = jnp.mean(jax.vmap(lambda r, t: jnp.abs(dq.trace(r@t)), in_axes=(0,0))(finals, exp_code_dm))

   # add measurement noise to fidelity calculation
    measurement_noise = 0*np.random.uniform(-0.01, 0.01)
    avg_gate_fidelity_loss = np.clip(utl.compute_fidelity_loss(evo_result) + measurement_noise, 0, 1)

    loss = 0
    cnt=0

    loss += 1*avg_gate_fidelity_loss
    cnt+=1

    return loss


In [45]:
gen_pulse = optimizer.generate_pulse()
print("Generated pulse shape:", gen_pulse.shape)

# This gives the target or best possible scenario after SPSA optimization
print('\nfidelity without parameter deviations and losses:', 1 - loss_calculator_func(gen_pulse, my_params))
print('fidelity with losses but no parameter deviation:', 1 - loss_calculator_func_mesolve(gen_pulse, my_params))

# This is the starting point for SPSA: shows how much deviation we have
print('\nFidelity in experiment without losses before SPSA optimization:', 1 - spsa_test_loss_calculator(gen_pulse, my_test_params))
print('Fidelity in experiment with losses before SPSA optimization:', 1- spsa_test_loss_calculator_mesolve(gen_pulse, my_test_params))

Generating pulse from the trained model...
Generated pulse shape: (2, 1000)

fidelity without parameter deviations and losses: 0.99228185
fidelity with losses but no parameter deviation: 0.96777976

Fidelity in experiment without losses before SPSA optimization: 0.974184
Fidelity in experiment with losses before SPSA optimization: 0.95233315


In [46]:
optimizer.fine_tune_hardware_spsa_all(
    test_loss_calculator=spsa_test_loss_calculator_mesolve,
    test_params=my_test_params,
    layer_name='final_hidden_layer', # Specify the layer to tune
    spsa_steps=1000,                 # NOTE: May need more steps
    a=0.01,                        # NOTE: May need a smaller step size
    c=0.01,
    update_plot_every=50,
    target_fidelity_loss=1e-3
)


--- Starting Hardware Fine-tuning (SPSA on ALL weights of layer: final_hidden_layer) ---
Targeting 260 parameters in layer 'final_hidden_layer'.
Initial loss before SPSA: 0.047667


SPSA Tuning (final_hidden_layer): 100%|██████████| 1000/1000 [14:56<00:00,  1.11it/s]


Optimization finished. Restoring best parameters found with loss: 0.029824


In [34]:
# optimizer.fine_tune_hardware_spsa(
#     test_loss_calculator=spsa_test_loss_calculator,
#     test_params=my_test_params,
#     spsa_steps=5000,
#     a=0.01,  # SPSA step size
#     c=0.01,  # SPSA perturbation size
#     update_plot_every=10,
#     target_fidelity_loss=1e-3
# )

# print("\nSPSA simulation test complete.")

In [35]:
# optimizer.fine_tune_hardware_spsa_all(
#     test_loss_calculator=spsa_test_loss_calculator,
#     test_params=my_test_params,
#     layer_name='final_hidden_layer', # Specify the layer to tune
#     spsa_steps=1500,                 # NOTE: May need more steps
#     a=0.01,                        # NOTE: May need a smaller step size
#     c=0.01,
#     update_plot_every=10,
#     target_fidelity_loss=1e-3
# )

In [39]:
spsa_pulse = optimizer.generate_pulse()
print("Generated pulse shape:", spsa_pulse.shape)
print('Fidelity in experiment after SPSA optimization:', spsa_test_loss_calculator(spsa_pulse, my_test_params))

# Visually compare by plotting
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

time_array = my_params['tpulse'][:-1]

# Cavity Drive Plot
axs[0].plot(time_array, np.real(gen_pulse[0, :]), 'b-', label='sim (Re)')
axs[0].plot(time_array, np.imag(gen_pulse[0, :]), 'r-', label='sim (Im)')
axs[0].plot(time_array, np.real(spsa_pulse[0, :]), 'b--', label='SPSA (Re)')
axs[0].plot(time_array, np.imag(spsa_pulse[0, :]), 'r--', label='SPSA (Im)')
axs[0].set_title('Cavity Drive Comparison')
axs[0].set_ylabel('Amplitude')
axs[0].legend()
axs[0].grid(True)

# Qubit Drive Plot
axs[1].plot(time_array, np.real(gen_pulse[1, :]), 'b-', label='sim (Re)')
axs[1].plot(time_array, np.imag(gen_pulse[1, :]), 'r-', label='sim (Im)')
axs[1].plot(time_array, np.real(spsa_pulse[1, :]), 'b--', label='SPSA (Re)')
axs[1].plot(time_array, np.imag(spsa_pulse[1, :]), 'r--', label='SPSA (Im)')
axs[1].set_title('Qubit Drive Comparison')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Amplitude')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

Generating pulse from the trained model...
Generated pulse shape: (2, 1000)
Fidelity in experiment after SPSA optimization: 0.015979666


In [ ]:
# Save trained model weights

directory = '/Users/saswataroy/OptimalControl/pulses/'
base_filename = 'spsa_trained_bin_X_gate'

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

weights_filepath = directory + base_filename +'_'+ timestamp + '.weights.h5'
params_filepath = directory + base_filename + '_'+ timestamp + '.npz'

optimizer.save_model_weights(weights_filepath)
np.savez(params_filepath, **my_test_params)
print(repr(weights_filepath))
print(repr(params_filepath))

print("\n Model trained on hardware is saved, ready for experiments")

In [ ]:
# Load model weights

directory = '/Users/saswataroy/OptimalControl/pulses/'
optimizer = HardwareAwareOptimizer(my_test_params)
optimizer.load_model_weights(directory + 'SPSA_tuned_model.weights.h5')
print('\n SPSA-tuned model loaded, proceed to experiments')

## Analysis: 

### Evolution and pulse:

In [ ]:
optimizer2 = HardwareAwareOptimizer(my_params)
optimizer.load_model_weights(directory + 'simulation_trained_bin_X_2025-09-02_13-20-53.weights.h5')
optimizer2.load_model_weights(directory + 'simulation_trained_bin_X_robust_2025-09-03_09-22-06.weights.h5')

gen_pulse = optimizer.generate_pulse()
rob_pulse = optimizer2.generate_pulse()

Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_2025-09-02_13-20-53.weights.h5
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_robust_2025-09-03_09-22-06.weights.h5
Generating pulse from the trained model...
Generating pulse from the trained model...


In [ ]:
print(loss_calculator_func(gen_pulse, my_params))
print(loss_calculator_func(rob_pulse, my_params))

0.0010128518
0.9749402


In [ ]:
# Compare the evolution before and after SPSA: 
my_test_params = my_params.copy()
det_qb = 0
det_osc = 0

e_ops_evo = []
e_ops_evo.extend([(dq.tensor(dq.proj(dq.basis(ncav, i)), dq.proj(dq.basis(ntr,0)) + dq.proj(dq.basis(ntr,1))  )) for i in range(ncav)])
e_ops_qb = []
e_ops_qb.extend([dq.tensor(idcav, dq.proj(dq.basis(ntr, i))) for i in range(ntr)])

my_params['e_ops_qb'] = e_ops_qb
my_params['e_ops_evo'] = e_ops_evo

T1q = 5000
T2q = 5000
Tphiq = (1/T2q - 1/(2*T1q))**(-1)

T1c = 20000
T2c = 20000
Tphic = (1/T2c - 1/(2*T1c))**(-1)

c_ops = [
        jnp.sqrt(1/T1q)*dq.tensor(idcav, t),
        jnp.sqrt(1/Tphiq)*dq.tensor(idcav, tdag @ t),
        jnp.sqrt(1/T1c)*dq.tensor(a, idtr), 
        jnp.sqrt(1/Tphic)*dq.tensor(adag @ a, idtr),
        ]

exp_state = [dq.tensor(dq.basis(ncav,2), dq.basis(ntr,0))]
exp_code_dm_list = [psi @ psi.dag() for psi in exp_state]
exp_code_dm = dq.stack(exp_code_dm_list)

my_params['T1q'] = T1q
my_params['T2q'] = T2q
my_params['T1c'] = T1c
my_params['T2c'] = T2c
my_params['c_ops'] = c_ops
my_params['exp_state'] = exp_state
my_params['exp_code_dm'] = exp_code_dm

def compute_evolution_mesolve(amps, params):
    """
    Compute the evolution of the system given the pulse amplitudes and parameters.
    """
    
    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # Add noise to the drive
    # iq_gain_errors = 1.0 + np.random.uniform(-0.001, 0.001, size=(4,))
    # amps_real_noisy = np.real(amps) * np.array([[iq_gain_errors[0]], [iq_gain_errors[2]]])
    # amps_imag_noisy = np.imag(amps) * np.array([[iq_gain_errors[1]], [iq_gain_errors[3]]])

    # noisy_amps = amps_real_noisy + 1j * amps_imag_noisy
    noisy_amps = amps

    # Add detuning to the drives
    qb_det_phi = det_qb * params['tpulse'][:-1]
    osc_det_phi = det_osc * params['tpulse'][:-1]
    det_osc_drive = noisy_amps[0,:] * jnp.exp(-1j*osc_det_phi)
    det_qb_drive = noisy_amps[1,:] * jnp.exp(-1j*qb_det_phi)

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_osc_drive)), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_osc_drive)), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(det_qb_drive)), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(det_qb_drive)), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.mesolve(H, rho0=psi0, tsave=tsave, jump_ops = c_ops, exp_ops = exp_ops, options = options, method = solver)
    
    prob_sim = np.zeros((params['ncav'], len(params['tsave'])))
    prob_qb_sim = np.zeros((params['ntr'], len(params['tsave'])))
    
    for i in range(evo_result.states.shape[1]):
        for j in range(ncav):
            prob_sim[j, i] = np.real(dq.ptrace(evo_result.states[0, i], dims = (ncav, ntr), keep = (0,)).to_jax()[j,j])

    for i in range(evo_result.states.shape[1]):
        for j in range(ntr):
            prob_qb_sim[j,i] = np.real(dq.ptrace(evo_result.states[0, i], dims = (ncav, ntr), keep = (1,)).to_jax()[j,j])

    return params['tsave'], prob_sim, prob_qb_sim

tsave, prob_sim_gen, prob_qb_sim_gen = compute_evolution_mesolve(gen_pulse, my_test_params)
_, prob_sim_spsa, prob_qb_sim_spsa = compute_evolution_mesolve(spsa_pulse, my_test_params)

KeyboardInterrupt: 

In [ ]:
colors = [
    "#0058A3",
    "#008C6E",
    "#D47A17",
    "#B95D99",
    "#6E6E6E",
    "#7A4A91",
    "#4A89C0",
    # "#C04851",
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey='row')

# --- Top-Left: Cavity Population (Generated Pulse) ---
ax = axes[0, 0]
for j, c in enumerate(colors):
    ax.plot(tsave, prob_sim_gen[j, :], '-', label=f'$P_{j}$', color=c)
ax.set_title('Cavity Population (Pre-SPSA)')
ax.set_ylabel('Population')
ax.legend()

# --- Top-Right: Cavity Population (SPSA Pulse) ---
ax = axes[0, 1]
for j, c in enumerate(colors):
    ax.plot(tsave, prob_sim_spsa[j, :], '-', label=f'$P_{j}$', color=c)
ax.set_title('Cavity Population (Post-SPSA)')
ax.legend()

# --- Bottom-Left: Qubit Population (Generated Pulse) ---
ax = axes[1, 0]
for j in range(ntr):
    ax.plot(tsave, prob_qb_sim_gen[j, :], '-', label=f'$P_{j}$', color=colors[j])
ax.set_title('Qubit Population (Pre-SPSA)')
ax.set_ylabel('Population')
ax.set_xlabel('Time ($\\mu s$)')
ax.legend()

# --- Bottom-Right: Qubit Population (SPSA Pulse) ---
ax = axes[1, 1]
for j in range(ntr):
    ax.plot(tsave, prob_qb_sim_spsa[j, :], '-', label=f'$P_{j}$', color=colors[j])
ax.set_title('Qubit Population (Post-SPSA)')
ax.set_xlabel('Time ($\\mu s$)')
ax.legend()

# Apply grids to all subplots
for ax_row in axes:
    for ax in ax_row:
        ax.grid(True, which='major', linestyle='-', linewidth='0.6', color='gray')
        ax.grid(True, which='minor', linestyle=':', linewidth='0.5', color='lightgray')
        ax.minorticks_on() 
        # ax.axhline(1, linestyle='--', color='k', alpha=0.5)

plt.tight_layout()

NameError: name 'prob_sim_spsa' is not defined

### Robustness check:

In [60]:
directory = 'C://Users//Fatem//SR_optctrl//Pulses//'

optimizer = HardwareAwareOptimizer(my_params)
optimizer.load_model_weights(directory + 'simulation_trained_bin_X_2025-09-02_13-20-53.weights.h5')

optimizer2 = HardwareAwareOptimizer(my_params)
optimizer2.load_model_weights(directory + 'simulation_trained_bin_X_robust_ordinary_2025-09-06_22-45-17.weights.h5')

Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_2025-09-02_13-20-53.weights.h5
Initializing Optimizer...
Neural network model created.
Model weights loaded successfully from C://Users//Fatem//SR_optctrl//Pulses//simulation_trained_bin_X_robust_ordinary_2025-09-06_22-45-17.weights.h5


In [7]:
def loss_calculator_func(amps, params):

    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[0,:])), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[0,:])), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[1,:])), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[1,:])), -1j * dq.tensor(idcav, (t - tdag)))
    H = params['H0'] + Hcr + Hci + Htr + Hti

    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    evo_result = dq.sesolve(H, psi0, tsave, exp_ops = exp_ops, options = options, method = solver)
    
    avg_gate_fidelity_loss = utl.compute_fidelity_loss(evo_result)

    loss = 0
    cnt=0

    loss += 1*avg_gate_fidelity_loss
    cnt+=1

    return loss

In [23]:
rob_pulse = optimizer.generate_pulse()
loss_calculator_func(rob_pulse, my_params)

Generating pulse from the trained model...


Array(0.00515553, dtype=float32)

In [61]:
def chi_robustness(amps, dets, params):

    """
    Evaluate the robustness of the pulse to chi(dispersive coupling) detuning by computing the average gate fidelity
    for different detuning values.

    Parameters:
    amps (ndarray): Optimized amplitude of shape (2, ntpulse-1) to be used to simulate the system.
    dets (ndarray): Detuning values for the qubit frequency to be used to simulate the system.
                    Ideally, this should have both negative and positive values.
                    Each value of det will be used for a different simulation with an addition of
                    det[i] * dq.tensor(a @ adag, t @ tdag) to params['H0'].
    params (dict): The usual params dictionary with all necessary parameters for simulation.

    Returns:
    ndarray: Average gate fidelity for binomial gate operations for the different detuning values,
             same shape as dets.
    """

    fids = np.empty(len(dets))

    a, adag = dq.destroy(params['ncav']), dq.create(params['ncav'])
    t, tdag = dq.destroy(params['ntr']), dq.create(params['ntr'])
    idcav = dq.eye(params['ncav'])
    idtr = dq.eye(params['ntr'])
    
    if params['osc_drive'] == 'linear':
        osc_pow = 1
    elif params['osc_drive']=='squeeze':
        osc_pow = 2

    # time-dependent Hamiltonian
    # (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
    Hcr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[0,:])), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
    Hci = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[0,:])), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
    Htr = dq.pwc(params['tpulse'], jnp.real(jnp.real(amps[1,:])), dq.tensor(idcav, t+tdag))
    Hti = dq.pwc(params['tpulse'], jnp.real(jnp.imag(amps[1,:])), -1j * dq.tensor(idcav, (t - tdag)))
    
    orig_H0 = params['H0'] + Hcr + Hci + Htr + Hti
    options = dq.Options(progress_meter = None)
    solver = dq.method.Tsit5(max_steps = int(1e9))

    for i, det in enumerate(dets): 
        H = orig_H0 + det * dq.tensor(a @ adag, t @ tdag)
        evo_result = dq.sesolve(H, psi0, tsave, exp_ops = exp_ops, options = options, method = solver)        
        avg_gate_fidelity_loss = utl.compute_fidelity_loss(evo_result)
        fids[i] = 1 - avg_gate_fidelity_loss

    return fids

In [62]:
dets = [0]
print(chi_robustness(rob_pulse, dets, my_params))
print(1 - loss_calculator_func(rob_pulse, my_params))
print(1-robust_loss_calculator(rob_pulse, my_params))

[0.99884081]
0.9988408


NameError: name 'robust_loss_calculator' is not defined

In [63]:
### Check robustness of the fine-tuned pulse: 
chi_dets = np.linspace(-0.04, 0.04, 11)*(2*jnp.pi)

rob_pulse = optimizer2.generate_pulse()
rob_fid = chi_robustness(rob_pulse, chi_dets, my_params)

gen_pulse = optimizer.generate_pulse()
gen_fid = chi_robustness(gen_pulse, chi_dets, my_params)

Generating pulse from the trained model...
Generating pulse from the trained model...


In [64]:
plt.figure()
plt.plot(chi_dets/(2*jnp.pi), rob_fid, 'o-', label='Robust Pulse')
plt.plot(chi_dets/(2*jnp.pi), gen_fid, 's--', label='Fidelity Pulse')
plt.legend()
plt.xlabel('Chi Detuning (MHz)')
plt.ylabel('Fidelity')

Text(0, 0.5, 'Fidelity')

In [52]:
print(gen_fid)
print(chi_dets/(2*jnp.pi))

[0.73793972 0.81619304 0.8846668  0.93899035 0.97533107 0.99086517
 0.98416179 0.95543277 0.90657848 0.84103799 0.76343006]
[-0.075 -0.06  -0.045 -0.03  -0.015  0.     0.015  0.03   0.045  0.06
  0.075]


### ET check:

In [40]:
amps = optimizer.generate_pulse()

psi0 = [dq.tensor(L0, dq.basis(ntr, 0)), 
        dq.tensor(L1, dq.basis(ntr, 0)),
        dq.tensor(plus_X, dq.basis(ntr, 0)),
        dq.tensor(minus_X, dq.basis(ntr, 0)),
        dq.tensor(plus_Y, dq.basis(ntr, 0)),
        dq.tensor(minus_Y, dq.basis(ntr, 0))
]

exp_ops = [dq.tensor(dq.proj(L1), dq.proj(dq.basis(ntr, 0))), 
           dq.tensor(dq.proj(L0), dq.proj(dq.basis(ntr, 0))),
           dq.tensor(dq.proj(plus_X), dq.proj(dq.basis(ntr, 0))), 
           dq.tensor(dq.proj(minus_X), dq.proj(dq.basis(ntr, 0))),
           dq.tensor(dq.proj(minus_Y), dq.proj(dq.basis(ntr, 0))), 
           dq.tensor(dq.proj(plus_Y), dq.proj(dq.basis(ntr, 0)))
]

a, adag = dq.destroy(my_params['ncav']), dq.create(my_params['ncav'])
t, tdag = dq.destroy(my_params['ntr']), dq.create(my_params['ntr'])
idcav = dq.eye(my_params['ncav'])
idtr = dq.eye(my_params['ntr'])

if my_params['osc_drive'] == 'linear':
        osc_pow = 1
elif my_params['osc_drive']=='squeeze':
        osc_pow = 2

# time-dependent Hamiltonian
# (sum of  piece-wise constant Hamiltonians and of the static Hamiltonian)
Hcr = dq.pwc(my_params['tpulse'], jnp.real(jnp.real(amps[0,:])), dq.tensor(dq.powm(a,osc_pow) + dq.powm(adag,osc_pow), idtr))
Hci = dq.pwc(my_params['tpulse'], jnp.real(jnp.imag(amps[0,:])), -1j * dq.tensor((dq.powm(a,osc_pow) - dq.powm(adag,osc_pow)), idtr))
Htr = dq.pwc(my_params['tpulse'], jnp.real(jnp.real(amps[1,:])), dq.tensor(idcav, t+tdag))
Hti = dq.pwc(my_params['tpulse'], jnp.real(jnp.imag(amps[1,:])), -1j * dq.tensor(idcav, (t - tdag)))
H = my_params['H0'] + Hcr + Hci + Htr + Hti

options = dq.Options(progress_meter = None)
solver = dq.method.Tsit5(max_steps = int(1e9))

evo_result = dq.sesolve(H, psi0, tsave, exp_ops = exp_ops, options = options, method = solver)

avg_gate_fidelity_loss = utl.compute_fidelity_loss(evo_result)

print(avg_gate_fidelity_loss)

Generating pulse from the trained model...
0.009472182


In [41]:
proj_even = sum(
    dq.tensor(
        dq.proj(dq.basis(my_params['ncav'], i)),
        dq.proj(dq.basis(my_params['ntr'], 0))
    )
    for i in range(my_params['ncav'])
    if i % 2 == 0
)

proj_odd = sum(
    dq.tensor(
        dq.proj(dq.basis(my_params['ncav'], i)),
        dq.proj(dq.basis(my_params['ntr'], 0))
    )
    for i in range(my_params['ncav'])
    if i % 2 == 1
)

T1q = 80

T2q = 80
Tphiq = (1/T2q - 1/(2*T1q))**(-1)

T1c = 200
T2c = 200
Tphic = (1/T2c - 1/(2*T1c))**(-1)


c_ops = [jnp.sqrt(1/T1q)*dq.tensor(idcav, t),
        jnp.sqrt(1/Tphiq)*dq.tensor(idcav, tdag @ t),
        jnp.sqrt(1/T1c)*dq.tensor(a, idtr), 
        jnp.sqrt(1/Tphic)*dq.tensor(adag @ a, idtr),
        ]

options = dq.Options(progress_meter = None)
solver = dq.method.Tsit5(max_steps = int(1e9))

evo_result = dq.mesolve(H, rho0 = psi0, tsave = my_params['tsave'], jump_ops = c_ops, method = solver, options=options)

In [42]:
def sqrtm_jax(A, tol=1e-5):
    """
    Compute the matrix square root using JAX.

    Parameters:
    A (jnp.ndarray or DenseQArray): Input matrix.
    tol (float): Tolerance below which eigenvalues are considered zero.

    Returns:
    jnp.ndarray: Matrix square root of the input matrix.
    """
    if isinstance(A, dq.qarrays.dense_qarray.DenseQArray):
        A = jnp.array(A.to_jax())  # Convert DenseQArray to JAX array

    # Ensure input matrix is valid
    if jnp.any(jnp.isnan(A)) or jnp.any(jnp.isinf(A)):
        raise ValueError("Input matrix contains NaNs or infinities")

    # Ensure input matrix is square
    if A.ndim != 2 or A.shape[0] != A.shape[1]:
        raise ValueError("Input must be a square matrix")

    # Ensure input matrix is Hermitian
    if not jnp.allclose(A, A.conj().T):
        raise ValueError("Input matrix must be Hermitian")

    # Compute the matrix square root
    eigvals, eigvecs = jnp.linalg.eigh(A)

    # Set small negative eigenvalues to zero
    eigvals = jnp.where(eigvals < tol, 0, eigvals)

    sqrt_eigvals = jnp.sqrt(eigvals)
    sqrt_A = eigvecs @ jnp.diag(sqrt_eigvals) @ jnp.conj(eigvecs.T)

    return sqrt_A

def den_fidelity(rho, sigma):
    """Compute the fidelity between two density matrices."""
    
    sqrt_rho = sqrtm_jax(rho)
    product = sqrt_rho @ sigma @ sqrt_rho
    sqrt_product = sqrtm_jax(product)

    return jnp.real(jnp.trace(sqrt_product)) ** 2

In [43]:
target_state = [dq.tensor(L1, dq.basis(my_params['ntr'], 0)), 
                dq.tensor(L0, dq.basis(my_params['ntr'], 0)), 
                dq.tensor(plus_X, dq.basis(my_params['ntr'], 0)), 
                dq.tensor(minus_X, dq.basis(my_params['ntr'], 0)), 
                dq.tensor(minus_Y, dq.basis(my_params['ntr'], 0)), 
                dq.tensor(plus_Y, dq.basis(my_params['ntr'], 0))
]
target_den = [target_state[i] @ target_state[i].dag() for i in range(len(target_state))]
recovery = dq.tensor( L1 @ E1.dag() + L0 @ E0.dag() , dq.eye(my_params['ntr']))

In [44]:
print(sum([den_fidelity(evo_result.states[i,-1], target_den[i]) for i in range(len(target_state))])/len(target_state))

0.96793246


In [45]:
print(sum([den_fidelity(dq.unit(proj_even @ evo_result.states[i,-1] @ proj_even), target_den[i]) for i in range(len(target_state))])/ len(target_state))

0.9921475


In [46]:
print(sum([den_fidelity(dq.unit(recovery @ dq.unit(proj_odd @ evo_result.states[i,-1] @ proj_odd) @ recovery.dag()), target_den[i]) for i in range(len(target_state))])/ len(target_state))

0.8841645
